# Pre-process and write to npz for GNN training and testing

In [14]:
import re
import numpy as np
import random

re_int_sci = r'[-\d\.]+e?[-+\d]*'
re_sci = r'[+-]?\d+\.\d+e[+-]?[\d]+'

## Concrete-2D-CI particle types based on particle index
PID_TO_TYPE = {1:   0,   # Seg1
               2:   0,   # Seg2
               3:   0,   # Seg3
               101: 1,   # Anchor_plate1
               102: 1,   # Anchor_plate2
               103: 1,   # Anchor
               104: 2,   # Tendon
               201: 3,   # Hammer
               211: 3,   # Roller
               221: 3,   # Plate_top
               222: 3,   # Plate_bot
               301: 4,   # Rebar_seg1
               302: 4,   # Rebar_seg2
               303: 4,   # Rebar_seg3
               401: 4,   # Stirrup_seg1
               402: 4,   # Stirrup_seg2
               403: 4,   # Stirrup_seg3
              }

def parse_segment_simulation(file):

    '''
    Extract info from LSDYNA txt.

    Input: text file, extracted with the following steps:
    1. Load d3plot file
    2. Output the 'Element' (eid, pid)
    3. Output 'Element Centroid, Volume' by appending all steps (pos)
    4. Select "effective plastic strain" and unselect all beam parts
    5. Output 'Element results' amd append all (eps)
    6. Select 'axial strain', select beam parts, unselect all other solid parts
    4. Output 'Element results' and append all (axs)

    The resulted txt file contains:
    1. The init particle id (eid) with part id (pid)
    2. For each timestep:
        2.1 solid particle position (pid,x,y,z)
        2.2 beam particle position (pid,x,y,z)
    3. For each timestep, the solid particle strain (eps) (pid, eps)
    4. And then for each timestep, the beam particle strain (axs) (pid, eps)
    Note if element errosion is adopted, the num_particle per timestep may vary,
    but eliminated particles are consistent between position and strain

    Output: np arrays
        tracjectory: (timesteps, num_particles, 3), 
        particle_types: (num_particles),  # if without errosion
        strains: (timesteps, num_particles).       
    '''

    with open(file, 'r') as f:
        lines = f.readlines()

    pos_lines_start, end_lines = [], []
    solid_strain_lines_start, beam_strain_lines_start = [], []
    for idx, line in enumerate(lines):
        if line.startswith("*ELEMENT_SOLID"):
            type_line_start = idx
        elif line.startswith("$Eid, X, Y, Z, Volume"):
            pos_lines_start.append(idx)
        elif line.startswith("$RESULT OF Effective Plastic Strain (Unaveraged)"):
            solid_strain_lines_start.append(idx)
        elif line.startswith("$RESULT OF Axial Strain (Unaveraged)"):  
            beam_strain_lines_start.append(idx)
        elif line.startswith("*END"):  # $NODAL_RESULTS,(1d) *INITIAL_VELOCITY_NODE(2d)
            end_lines.append(idx)

    type_line_end = end_lines[0]    # the first *END is for particle type
    num_timesteps = len(pos_lines_start)
    pos_lines_end = end_lines[1:num_timesteps+1]
    solid_strain_lines_end = end_lines[num_timesteps+1:2*num_timesteps+1]
    beam_strain_lines_end = end_lines[2*num_timesteps+1:]

    # Extact particle types
    particle_types = []
    eids = []
    for line in lines[type_line_start:type_line_end]:
        num_str = re.findall(re_int_sci, line)  # Regular expression findign integers
        if len(num_str) >= 4:
            eid = int(num_str[0])
            pid = int(num_str[1])
            particle_type = PID_TO_TYPE[pid]
            eids.append(eid)
            particle_types.append((eid, particle_type))
    particle_types = np.array(particle_types).astype(int)

    # Extact particle positions 
    trajectory = []
    for line_start, line_end in zip(pos_lines_start, pos_lines_end):
        pos_lines = lines[line_start+1:line_end]   # lines that contains positions in one time step
        pos_one_step = []
        for line in pos_lines:
            num_str = re.findall(re_int_sci, line)  # Regular expression findign scitific numbers
            if len(num_str) >= 4:
                pos = [float(x) for x in num_str[:4]] # [eid, x, y, z]
                pos = tuple(pos)
                pos_one_step.append(pos)
        trajectory.append(pos_one_step) 
    trajectory = np.array(trajectory)

    # Extract effective plastic strain for solids (eps)
    solid_strains = []
    for line_start, line_end in zip(solid_strain_lines_start, solid_strain_lines_end):
        strain_lines = lines[line_start+1:line_end]   # lines that contains positions in one time step
        strains_one_step = []
        for line in strain_lines:
            num_str = re.findall(re_int_sci, line)  # Regular expression findign scitific numbers
            if len(num_str) == 2:
                num = [float(x) for x in num_str]
                strains_one_step.append(tuple(num))
        solid_strains.append(strains_one_step)
    solid_strains = np.array(solid_strains).astype(float)

    # Extract axial strain for beams (axs)
    beam_strains = []
    for line_start, line_end in zip(beam_strain_lines_start, beam_strain_lines_end):
        strain_lines = lines[line_start+1:line_end]   # lines that contains positions in one time step
        strains_one_step = []
        for line in strain_lines:
            num_str = re.findall(re_int_sci, line)  # Regular expression findign scitific numbers
            if len(num_str) == 2:
                num = [float(x) for x in num_str]
                strains_one_step.append(tuple(num))
        beam_strains.append(strains_one_step)
    beam_strains = np.array(beam_strains).astype(float)

    # Concatenate solid and beam 
    strains = np.concatenate((solid_strains, beam_strains), axis=1)

    # Sort based on eid
    idx = particle_types[:, 0].argsort()
    particle_types = particle_types[idx, 1]

    idx = strains[0, :, 0].argsort()
    strains = strains[:, idx, 1]

    idx = trajectory[0, :, 0].argsort()
    trajectory = trajectory[:, idx, 1:]

    print('trajectory: ', trajectory.shape)
    print('particle_types: ', particle_types.shape)
    print('strains :', strains.shape)

    return trajectory, particle_types, strains


if __name__ == "__main__":
    pass

In [20]:
import numpy as np
import glob
import json
import random
import math
import pathlib
       
dataset = 'segment_beam'
in_dir = f'/home/jovyan/work/data_temp/{dataset}/DYNA/'
out_dir = f'/home/jovyan/work/data_temp/{dataset}/'
pathlib.Path(out_dir).mkdir(parents=True, exist_ok=True)

# Grab all simulation cases from corresponding data folder
simulations = glob.glob(in_dir +'*')
random.shuffle(simulations)

## Larger step size leads to shorter trajectory and hence better rollout performance
## But lower precision of the simulation
## Current simulation are of absolute time 1000 ms
## Step size=1 means 100 steps, each of which 10 ms
STEP_SIZE = 3

## Initialisation placeholders for data
n_trajectory = len(simulations)
ds_train, ds_valid, ds_test = {}, {}, {}
vels = np.array([]).reshape(0, 3)
accs = np.array([]).reshape(0, 3)
strain_stats = np.array([])
file_train, file_valid, file_test = [], [], []

## Main loop for data extraction
for idx, simulation in enumerate(simulations):
    print(f"{idx}/{n_trajectory} Reading {simulation}...")
    positions, particle_types, strains = parse_segment_simulation(simulation)
    dim = positions.shape[-1]
    
    positions = positions[::STEP_SIZE, :, :]
    
    strains = strains[::STEP_SIZE, :]
       
    # print for debug
    print(f"Dim: {dim}")
    print(f"Position min:{positions.min(axis=(0,1))}, max:{positions.max(axis=(0,1))}")
    print(f"Strain min:{strains.min(axis=(0,1))}, max:{strains.max(axis=(0,1))}")
    print(f"Shape, pos: {positions.shape}, types: {particle_types.shape}, strain: {strains.shape}")
    print(f"Unique particle types: {np.unique(particle_types)}")
    
    # Data splits: train(80%), valid(10%), test(10%)
    key = f'trajectory_{idx}' 
    if True:
        print('to train')
        ds_train[key] = [positions, particle_types, strains]
        file_train.append(simulation)
    if True:
        print('to valid')
        ds_valid[key] = [positions, particle_types, strains]
        file_valid.append(simulation)
    if True:
        print('to test')
        ds_test[key] = [positions, particle_types, strains]
        file_test.append(simulation)
        
    # Extract Vel and Acc statistics
    # positions of shape [timestep, particles, dimensions]
    vel_trajectory = positions[1:,:,:] - positions[:-1,:,:]
    acc_trajectory = vel_trajectory[1:,:,:]- vel_trajectory[:-1,:,:]
    
    vels = np.concatenate((vels, vel_trajectory.reshape(-1, dim)), axis=0)
    accs = np.concatenate((accs, acc_trajectory.reshape(-1, dim)), axis=0)

# Extract vel, acc statistics for normalisation
vel_mean, vel_std = list(vels.mean(axis=0)), list(vels.std(axis=0))
acc_mean, acc_std = list(accs.mean(axis=0)), list(accs.std(axis=0))

# Save datasets in numpy format
np.savez(out_dir + 'train.npz', **ds_train)
np.savez(out_dir + 'valid.npz', **ds_valid)
np.savez(out_dir + 'test.npz', **ds_test)

print(f"{len(ds_train)} trajectories saved to train.npz.")
print(f"{len(ds_valid)} trajectories saved to valid.npz.")
print(f"{len(ds_test)}  trajectories saved to test.npz.")

# Save meta data
in_file = '/home/jovyan/share/gns_data/Concrete2D-C/metadata.json'
out_file = out_dir + 'metadata.json'

with open(in_file, 'r') as f:
    meta_data = json.load(f)

# In GNN, the suggested connection radius is 4.5r, or 5.625 mm (aounrd 20 neighbors)
# If R is 5 mm before normalization, 
meta_data['dim'] = 3
meta_data['default_connectivity_radius'] = 15 
meta_data['sequence_length'] = positions.shape[0]
meta_data['vel_mean'] = vel_mean
meta_data['vel_std'] = vel_std
meta_data['acc_mean'] = acc_mean
meta_data['acc_std'] = acc_std

meta_data['dt'] = 0.01 * STEP_SIZE
meta_data['bounds'] = [[-170, 170], [-250, 660], [-80, 1630]]
meta_data['file_train'] = file_train
meta_data['file_valid'] = file_valid
meta_data['file_test'] = file_test
print(meta_data)

with open(out_file, 'w') as f:
    json.dump(meta_data, f)

0/1 Reading /home/jovyan/work/data_temp/segment_beam/DYNA/seg_beam_100steps...
trajectory:  (101, 49814, 3)
particle_types:  (49814,)
strains : (101, 49814)
Dim: 3
Position min:[-161.34921  -237.68245   -68.810745], max:[ 161.47427  650.03168 1617.6865 ]
Strain min:-0.003369695, max:1.9999939
Shape, pos: (34, 49814, 3), types: (49814,), strain: (34, 49814)
Unique particle types: [0 1 2 3 4]
to train
to valid
to test
1 trajectories saved to train.npz.
1 trajectories saved to valid.npz.
1  trajectories saved to test.npz.
{'bounds': [[-170, 170], [-250, 660], [-80, 1630]], 'sequence_length': 34, 'default_connectivity_radius': 15, 'dim': 3, 'dt': 0.03, 'vel_mean': [-0.001834683072605822, 0.017270407932242788, -0.11164911523248897], 'vel_std': [0.04751214060138093, 1.0044909292099198, 0.20779822103955548], 'acc_mean': [-0.0005519265941803202, 0.06471453574952156, -0.0004543210862477105], 'acc_std': [0.06523576891911972, 0.3524579119577412, 0.09076977260142277], 'file_train': ['/home/jovyan/

# Modify metadata

In [ ]:
in_file = '/home/jovyan/share/gns_data/Fragment/metadata.json'
out_file = f'/home/jovyan/work/data_temp/fragment/Fragment/metadata.json'

with open(in_file, 'r') as f:
    meta_data = json.load(f)

meta_data['dim'] = 3
meta_data['bounds'] = [[-500, 500], [-1000, 1000], [0, 255]]

print(meta_data)

with open(out_file, 'w') as f:
    json.dump(meta_data, f)

# Test regular expression for number extraction

In [ ]:
import re

strs = ['20742   1.4952594e+03   -1.0499660e+02   1.6313647e-02   9.9995575e+02',
        '    32365   1.4051317e+00',
        '   10826       1   15757   15758   15784   15783   11311   11312   11338   11337',
        '$Total Solid element Volume =    7.5878880e+07'
       ]

pattern = r'[+-]?\d+\.\d+e[+-]?[\d]+'
for str in strs:
    print(re.findall(pattern, str))
    
pattern = r'[-\d\.]+e?[-+\d]*'
for str in strs:
    print(re.findall(pattern, str))

# Plot segment data

In [ ]:
import pickle

from absl import app
from absl import flags

from matplotlib import animation
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import os

def colormap(strains):
    """
    Create a colormap for each timestep based on strain
    """
    color_map = []
    for t in range(101):
        normalized_strain = mcolors.Normalize(
            vmin=0, 
            vmax=2
        )
        colormap = plt.get_cmap('jet')
        color_map.append(colormap(normalized_strain(strains[t])))
    return color_map

# Init figures
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

# Get boundary of simulation
xmin, ymin, zmin = trajectory.min(axis=(0,1))
xmax, ymax, zmax = trajectory.max(axis=(0,1))
ax.set_box_aspect([xmax-xmin, ymax-ymin, zmax-zmin])

plt.tight_layout()
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
# Fig creating function for 3d
def animate(i):
    print(f"Render step {i}/101...")

    fig.clear()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_box_aspect([xmax-xmin, ymax-ymin, zmax-zmin])
    color_map = colormap(strains)
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.set_xlim([float(xmin), float(xmax)])
    ax.set_ylim([float(ymin), float(ymax)])
    ax.set_zlim([float(zmin), float(zmax)])
    ax.scatter(trajectory[i, :, 0],
               trajectory[i, :, 1],
               trajectory[i, :, 2], 
               s=1, 
               color=color_map[i]
              )
    # rotate viewpoints angle little by little for each timestep
    ax.view_init(elev=20, azim=0, roll=0, vertical_axis='z')
    ax.grid(True, which='both')
    ax.set_title('LS-DYNA')
plt.tight_layout()
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

# Creat animation
ani = animation.FuncAnimation(
    fig, animate, frames=np.arange(0, 101, 10), interval=10)

ani.save(f'seg_beam.gif', dpi=100, fps=10, writer='Pillow')

In [17]:
print(xmin, xmax)
print(ymin, ymax)
print(zmin, zmax)

-161.35165 161.47925
-237.68259 650.07532
-68.812531 1617.7135
